# Object as source of rays and image loss

SourceObject
    an 'object' in the sense of optics that emits (really reflects) rays in all directions
    act as a source for the optical system
    properties: position, height

Ray:
    - object of origin?
    - absolute origin (object space coordinate of source)
    - during raytracing: origin / unit vector

TargetImage:
    - height
    - position

SourceDiffuse
    A diffuse light source that's not an object
    angular power function
    emits rays that are not associated with an object source

BeamLoss:
    Loss function for a beam of light spread,
    usefull to minimize beam spread

Object at infinity / image at infinity. ex: moon
    specific object
    specific image


OpticalObject
OpticalImage

OpticalObjectAtInfinity
OpticalImageAtInfinity

Moon / telescope example
    two rays coming from the same point on the object at infinity are //
    two rays coming from different points on the object are not //
    object at infinity doesn't have a size, but has angular size

In [ ]:
import torch
import torchlensmaker as tlm

class Optics(tlm.Module):
    def __init__(self):
        super().__init__()

        # lens diameter in mm
        lens_diameter = 60.

        # shape of the lens surface
        self.shape = tlm.CircularArc(lens_diameter, tlm.Parameter(torch.tensor(70.)))

        self.lens = tlm.SymmetricLens(self.shape, (1.0, 1.5), outer_thickness=5.)
        
        self.optics = tlm.Sequential(
            tlm.ObjectAtInfinity(beam_diameter=40, angular_size=15, angle=0),
            #tlm.PointSourceAtInfinity(beam_diameter=40, angle=7),
            tlm.Gap(10.),
            
            self.lens,
            
            tlm.Gap(60.0), # focal length
            #tlm.FocalPointLoss(),
            tlm.Image(height=10),
        )

    def forward(self, inputs):
        return self.optics(inputs)

optics = Optics()

tlm.render_plt(optics, sampling_info)
    
loss = tlm.loss(optics, sampling_info)